Copyright (c) 2019 [윤기태]

https://github.com/yoonkt200/python-data-analysis

[MIT License](https://github.com/yoonkt200/python-data-analysis/blob/master/LICENSE.txt)

# (가제) 파이썬 데이터 분석

-----

# 2.3) 영화 평점 예측하기

### 바로가기

- [1. MovieLens 데이터 살펴보기](#1.-MovieLens-데이터-살펴보기)
    - [MovieLens 데이터셋의 기본 정보]
    - [탐색적 데이터 분석]
        - 희소 매트릭스 데이터 시각화
        - 연도별/장르별로 etc.. 데이터 살펴보기
- [2. 수학적 기법을 활용한 평점 예측](#2.-수학적-기법을-활용한-평점-예측)
    - [SVD 기법 적용하기]
    - [아직 평가하지 않은 영화의 점수 예측하기]
- [3. 예측 결과 분석하기](#3.-예측-결과-분석하기)
    - [예측 모델의 평가]
    - [유저별/연도별/장르별 etc.. 가장 평점이 높은 영화]
    - [?!?@와 가장 유사한 영화 리스트]
- [내가 아직 보지 않은 이 영화의 평점은?](#내가-아직-보지-않은-이-영화의-평점은?)
- [부록 : 선배의 Summary](#부록-:-선배의-Summary)

-----

In [1]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# 1. MovieLens 데이터 살펴보기

### [MovieLens 데이터셋의 기본 정보]

In [11]:
# Data Source : https://grouplens.org/datasets/movielens

rating_file_path = "../data/ml-1m/ratings.dat"
movie_file_path = "../data/ml-1m/movies.dat"
user_file_path = "../data/ml-1m/users.dat"

rating_data = pd.io.parsers.read_csv(rating_file_path, 
                                     names=['user_id', 'movie_id', 'rating', 'time'], delimiter='::')
movie_data = pd.io.parsers.read_csv(movie_file_path, 
                                    names=['movie_id', 'title', 'genre'], delimiter='::')
user_data = pd.io.parsers.read_csv(user_file_path, 
                                   names=['user_id', 'gender', 'age', 'occupation', 'zipcode'], delimiter='::')

In [12]:
rating_data.head()

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [13]:
movie_data.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
user_data.head()

,user_id,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [64]:
import numpy as np
LA = np.linalg

a = np.array([[1, 3, 4], [5, 6, 9], [1, 2, 3], [7, 6, 8]])
print(a)
U, s, Vh = LA.svd(a, full_matrices=False)
print("-----")

print(s)
print(s[2:])

print("-----")
new_a = np.dot(U, np.dot(np.diag(s), Vh))
print(new_a)

print("-----")
s[2:] = 0
new_a = np.dot(U, np.dot(np.diag(s), Vh))
print(new_a)

[[1 3 4]
 [5 6 9]
 [1 2 3]
 [7 6 8]]
-----
[18.037639400729418   2.3436029200997703  0.388703231467832 ]
[0.388703231467832]
-----
[[1.0000000000000002 2.9999999999999973 3.9999999999999982]
 [4.999999999999998  5.999999999999999  8.999999999999996 ]
 [1.0000000000000007 1.9999999999999998 2.999999999999999 ]
 [6.999999999999996  5.999999999999999  7.9999999999999964]]
-----
[[1.022067551613378  2.7727630799783936 4.146513362128527 ]
 [4.980347401140943  6.202369348144838  8.869520262879636 ]
 [0.9978607695592145 2.0220283674916604 2.985796981480389 ]
 [7.011047834385008  5.886236774131434  8.07335002035323  ]]
